# Exercise - week 2: German translator

This should include a Gradio UI, streaming, use of the system prompt to add expertise, and the ability to switch between models. Bonus points if you can demonstrate use of a tool!

The assistant will transform your spoken English to text, then translate it German and speak it out. The image on the UI is just decoration. This exercise was created on MacOS, Python 3.13.

In [ ]:
# Install first PortAudio, in MacOS
# brew install portaudio


!pip install openai speechrecognition pyaudio


In [1]:
# imports

import os
import json
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr

In [ ]:
# Initialization

load_dotenv()

openai_api_key = os.getenv('OPENAI_API_KEY')
if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")
    
MODEL = "gpt-4o-mini"
openai = OpenAI()

In [3]:
system_message = """You are a highly skilled language translator specializing in translating English text to German. 
Your task is to accurately translate any English text provided by the user into German. 
Ensure that the translations are grammatically correct and contextually appropriate. 
If the user provides a phrase, sentence, or paragraph in English, respond with the equivalent translation in German.""" 

In [26]:
import speech_recognition as sr
from pydub import AudioSegment
from pydub.playback import play
import base64
from io import BytesIO
from PIL import Image


def recognize_speech(audio_file):
    recognizer = sr.Recognizer()
    with sr.AudioFile(audio_file) as source:
        audio = recognizer.record(source)
    try:
        text = recognizer.recognize_google(audio)
        return text
    except sr.UnknownValueError:
        return "Google Speech Recognition could not understand audio"
    except sr.RequestError as e:
        return f"Could not request results from Google Speech Recognition service; {e}"


def get_chatgpt_response(message):
    response = openai.chat.completions.create(
        model=MODEL,
        messages = 
            [{"role": "system", "content": system_message},
            {"role": "user", "content": message}],
        max_tokens=150
    )
    return response.choices[0].message.content.strip()

def process_audio(audio_file):
    text = recognize_speech(audio_file)
    if text:
        response = get_chatgpt_response(text)
        talker(response)
        return response
    return "Could not recognize speech."

def talker(message):
    response = openai.audio.speech.create(
      model="tts-1",
      voice="onyx",    # Also, try replacing onyx with alloy
      input=message
    )
    
    audio_stream = BytesIO(response.content)
    audio = AudioSegment.from_file(audio_stream, format="mp3")
    play(audio)

In [ ]:
# Create Gradio interface

# some image decoration to UI, just a static picture
image_path ="week2-exercise-translator-berlin.webp"

with gr.Blocks() as ui:
    gr.Interface(
        fn=process_audio,
        inputs=gr.Audio(type="filepath", label="Speak English. German translation in a moment:"),
        outputs="text",
        live=True, 
    )
    gr.Image(value=image_path, label="Das ist Berlin")
    
ui.launch(inbrowser=True)
